In [1]:
import os
import time
import pickle
import pandas as pd
from bot_utils import upload_data, manual_corruption, delete_generated_data, hash_message, verify_hash
from bot_utils import data_path, cloud_path, failure_reports_dir, data_owner_path
file_record_dir = r"D:\User Data\Desktop\User Revokeable Distributed Auditing\Cloud Service Provider\file_records_dir"

In [2]:
def auditing_process(data_path, cloud_path,meta_data):
    failure_records = {}
    for i,file in enumerate(os.listdir(cloud_path)):
        cloud_file_path = cloud_path+"\\"+file
        file_record = meta_data[file[:-4]]
        file_text = open(cloud_file_path).read()
        hash_text = "".join(file_record[0:3])+file_text+file_record[4]
        if not(verify_hash(hash_text, file_record[-1])):
            failure_records[file[:-4]] = [file[:-4], file_record[1], file_record[4]]
    return failure_records

In [3]:
def generate_failure_report(failure_records, n_files, per):
    report_path = failure_reports_dir +"\\"+"n_files_"+str(n_files)+"_FR_"+str(per)+".csv"
    files_id = list(failure_records.keys())
    versions = [failure_records[Id][1] for Id in files_id]
    time_stamps = [failure_records[Id][2] for Id in files_id]
    report = {"File_ID": files_id, "Version": versions, "Time_Stamp": time_stamps}
    pd_report = pd.DataFrame(report)
    csv_report = pd_report.to_csv(report_path)
    return report_path

In [4]:
def reverification(FR_path,file_record):
    fialure_records = pd.read_csv(FR_path)
    count = 0
    for file_id in fialure_records["File_ID"]:
        record = file_record[file_id]
        file_text = open(cloud_path+"\\"+file_id+".txt").read()
        hash_text = "".join(record[0:3])+file_text+record[4]
        if not(verify_hash(hash_text, file_record[-1])):
            count = count + 1
    return count
        
        
        

In [6]:
no_of_files = [10000,20000,30000,50000,100000]

for _ in range(10):
    for n_files in no_of_files:
        for per in [1,2,3,4,5]:
            upload_data(data_owner_path, n_files, 1, "data")
            print(n_files,"Files are sccessfully uploaded to the cloud.")

            manual_corruption(per, n_files, cloud_path, file_record_dir)
            print(per,"% Manual Corruption of file at cloud.")

            with open(data_path+"\\"+"output.pkl", "rb") as f:
                meta_data = pickle.load(f)
            delete_generated_data(data_path)

            print("Sequencial Auditing Process for",n_files,"reads\n\n")
            failure_records = auditing_process(data_path,cloud_path,meta_data)


            FR_path = generate_failure_report(failure_records,n_files,per)

            start = time.time()
            checked_files = reverification(FR_path,meta_data)
            end = time.time()
            print(checked_files,"files are reverified by Data owner")
            print(end-start,"Reverification Time.")
            delete_generated_data(cloud_path)
        break

10000 Files are sccessfully uploaded to the cloud.
1 % Manual Corruption of file at cloud.
Sequencial Auditing Process for 10000 reads


100 files are reverified by Data owner
0.04914736747741699 Reverification Time.


In [7]:
df_failure_report = pd.read_csv(FR_path)
df_failure_report.head()

,Unnamed: 0,File_ID,Version,Time_Stamp
0,0,file_1079,1,2024-08-05 18:06:19.603958
1,1,file_118,1,2024-08-05 18:06:20.064236
2,2,file_1195,1,2024-08-05 18:06:20.117489
3,3,file_1282,1,2024-08-05 18:06:20.390937
4,4,file_1372,1,2024-08-05 18:06:20.571026


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

